In [1]:
!pip install pathlib
!pip install pandas



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
from pandas import DataFrame
import pandas as pd
from pathlib import Path

def get_claims_data_from_folder(folder:Path,decision:str,negative_decision:str, max_number_to_process:int=float('inf'))-> list[dict]:
    ret_val=[]
    #We iterate over all the sub-folders in the folder and collect all of the files named ClaimHistory.json
    number_processed = 0
    for sub_folder in folder.iterdir():
        if (sub_folder.is_dir()):
            for file in sub_folder.iterdir():
                if file.name == "ClaimHistory.json":
                    with open(file, "r") as f:
                        claim_history = json.load(f)
                    #print (f"Found ClaimHistory.json for {sub_folder}")
                    #go through each claim in the claim history json and store the policy number, claim number, and status
                    for claim in claim_history:
                        policy_number = claim["policyNumber"]
                        claim_number = claim["id"]
                        status = claim["status"]
                        status_reason = claim["statusReason"]
                        #we then use the claim_number and see if there is a sub-folder within sub_folder named after the claim_number, if it does exist, we add the claim_number to the paid_claims list
                        if (sub_folder / str(claim_number)).exists():
                            ret_val.append({"policy_number": policy_number, "claim_number": claim_number, "status": status, "status_reason": status_reason, "in_data_set": True, "decision": decision})
                        else:
                            ret_val.append({"policy_number": policy_number, "claim_number": claim_number, "status": status, "status_reason": status_reason, "in_data_set": False, "decision": negative_decision})
                    number_processed += 1
                    if number_processed >= max_number_to_process:
                        print(f"Reached max number of claims to process: {max_number_to_process}")
                        break
    return ret_val

paid_folder = Path("/home/bobby/Blue Label Labs Dropbox/Bobby Gill/BL364 - Assurity/Client Data Dump - 04022025/INDIVIDUAL/PAID")

denied_folder = Path("/home/bobby/Blue Label Labs Dropbox/Bobby Gill/BL364 - Assurity/Client Data Dump - 04022025/INDIVIDUAL/DENIED")
output_file = Path("./res/outputs/ground_truth_claims_decisions/policy_claims_decisions.csv")

max_number_to_process = 10

paid_claims = get_claims_data_from_folder(paid_folder, "PAID", "DENIED")
print(f"Found {len(paid_claims)} paid claims")
unpaid_claims = get_claims_data_from_folder(denied_folder, "DENIED", "PAID")
print(f"Found {len(unpaid_claims)} denied claims")

claims_data = paid_claims + unpaid_claims

#convert to pandas dataframe
claims_data_df = pd.DataFrame(claims_data)

#output the paid claims to a csv file
claims_data_df.to_csv(output_file, index=False, header=True)



Found 182 paid claims
Found 143 denied claims
